# Crawler API

Crawler que consome a API do Twitter para coletar tweets presentes na timeline de um usuário.

Este notebook é um guia! Você vai precisar rodar localmente o código, mas não pule essa etapa, pois é muito importante entender como funciona a ferramenta e como manipular os dados que você coletar.

# Introdução

Para dar um pouco de contexto, vamos ver o motivo desse crawler ter sido criado. Ele foi modelado para coletar dados que 
identificassem como a fila da timeline do twitter se forma, por isso ele se concentrou em observar o próprio usuário logado em vez de diversos tweets espalhados pela rede.


# Configurando o Ambiente

O Twitter se tornou mais restrito quanto a quem pode ter uma chave de acesso à sua API. É necessário pedir uma conta de desenvolvedor na página [apps.twitter.com] e justificar o seu uso.

Entre na sua conta do Twitter e em seguida crie uma aplicação na página [https://developer.twitter.com/en/apps]. O nome e a descrição não importam muito, faça como quiser. Para preencher o campo 'website URL' use qualquer link, até um que não seja registrado. 

Infelizmente a burocracia não acabou e você precisa descrever qual é seu objetivo fazendo esse app, a boa notícia é o mínimo ser de apenas 100 caracteres.

Ao final do processo, você terá à disposição algumas credenciais que iremos usar adiante.

# Conectando à API do twitter

Para se conectar a API, você vai escrever as credenciais geradas pelo app num arquivo chamado "credentials.txt".

## Exemplo de credenciais

#### O arquivo precisa estar no formato abaixo com seus dados

Consumer_key = uQl2J9OpHkqXcBKdrNjU09UB0

Consumer_secret = FdGsRUWESM3wrQVF6nDrOgsgi99CpwOvylA3yBt1m6EeJn1btg

Acess_token = 772557411177623882-GtuCaR7jApm6BGUVsnjyGXQIiHfKjiy

Acess_token_secret = d5n1VTrHIULSHSTF9VueoBOUZIlzsN7nFWkZkn0Ap1dPE2


### Atenção!

As credenciais acima são apenas ilustrativas. Quando você gerar suas credenciais, JAMAIS compartilhe elas em lugar algum.


##  Bibliotecas

A biblioteca OAuth2 será usada para autenticar nossa conexão à API do twitter, o pymongo será usado para nos conectar ao banco mongo responsável por salvar nossos dados.



# JSON

Todos os tweets coletados serão salvos no formato JSON. Caso encontre alguma dificuldade em manipular ou entender o formato, confira esse artigo: https://pythonhelp.wordpress.com/2013/03/21/acessando-conteudo-via-apis-web-baseadas-em-json/


In [25]:
# -*- coding: utf-8 -*-
import re
import oauth2 as oauth
import json
import time
import codecs
import sys
import datetime
import pymongo

'''
__author__ = "Tiago Cruz de França"
__copyright__ = "Copyright 2018, UFRRJ"
__credits__ = ["Tiago França"]
__license__ = "GPL"
__version__ = "0.0.1"
__maintainer__ = "Tiago França, Edu Mangabeira"
__email__ = "tcruz.franca@gmail.com"
__status__ = "Test"
'''

'\n__author__ = "Tiago Cruz de França"\n__copyright__ = "Copyright 2018, UFRRJ"\n__credits__ = ["Tiago França"]\n__license__ = "GPL"\n__version__ = "0.0.1"\n__maintainer__ = "Tiago França, Edu Mangabeira"\n__email__ = "tcruz.franca@gmail.com"\n__status__ = "Test"\n'

# Acessando o mongoDB


Agora vai uma explicação rápida sobre o banco mongo, você pode pular essa parte por enquanto, pois só vai poder testar depois que todo o código for executado e os dados obtidos.

1) Para iniciar no seu terminal, basta digitar 'mongo'.

2) Primeiro é preciso ver quais bancos estão sendo usados na sua máquina, o comando a ser usado é 'show dbs'.

3) Para usar um banco: 'use {insira aqui um banco, sem chaves}'

4) Agora temos que selecionar uma coleção do nosso banco, para ver as coleções: 'show collections'.

5) Existem diferentes formas de acessar os dados, vou citar algumas.

5.1) 'db.{sua collection}.find()' mostra alguns tweets, é útil para verificar rapidamente erros em parâmetros. Também é possível filtrar a procura, para saber com mais detalhes veja em: https://docs.mongodb.com/manual/reference/method/db.collection.find/

5.2) 'db.{sua collection}.findOne()' mostra apenas um tweet e também pode filtrar a busca(recomendado nesse caso).

Se você deseja apenas um arquivo com a base de dados, volte para o seu terminal(crtl+C interrompe o processo mongo) e digite o seguinte comando:

##### Para obter .json
mongoexport --db {seu_banco} --collection {sua_coleção} --out {diretorio/nome_do_seu_arquivo}.json

##### Para obter .csv
mongoexport --db {seu_banco} --collection {sua_coleção} --type=csv --out {diretorio/nome_do_seu_arquivo}.csv

Existem opções mais específicas não citadas aqui, como obter apenas alguns campos da tabela. Aqui está a documentação do MongoDB, caso queria usar mais alguma função não apresentada aqui. https://docs.mongodb.com/manual/tutorial/getting-started/

## Definindo uma coleta

A partir daqui pode ficar um pouco complicado entender todo o código. O que é importante mesmo de forma mais direta se encontra a partir da função " *Salvartweets(data, collection, collection2, log, screen_name)* ". Caso encontre alguma dificuldade em entender o que a ferramenta faz, pule direto para lá.


Não se preocupe muito agora em entender a classe abaixo em detalhes, ela apenas será criada para nos ajudar a salvar um tweet com uma boa formatação no nosso banco de dados. Ela obtém um padrão de identificação entre cada coleta.

Ao longo do código pode ser que você esbarre em uma função ou outra que não aparente ter uma utilidade clara, por isso os pontos principais estarão bem destacados.

In [10]:
class SnapshotIdSingleton(object):

    snapshot = None
    aux = 0
    def __init__(self):
        if (self.aux == 0):
            self.__done__()
            self.__del__()

    def __del__(self):
        print("objeto destruido") 
   
    def __done__(self):
        print ("Por favor, use o metodo instance para criar uma instancia desta classe, ela e singleton")

    def getLastSnapShotIdFromMongo(self, collection):
        register = collection.find({},{"_id":0,"snapshot_id":1}).sort("snapshot_id",pymongo.DESCENDING).limit(1)
        register = register.next()
        return register["snapshot_id"]

    
    @classmethod 
    def instance(cls):
        if (cls.snapshot == None):
            cls.aux = 1
            cls.snapshot = SnapshotIdSingleton()
            cls.aux = 0

        return cls.snapshot

Mais uma vez não se preocupe, o registro de erros é apenas uma função para facilitar o desenvolvimento.

In [ ]:
def logError(log, msg):
    log.write(msg+"\n")

É uma função que vai ser usada ao longo do código para auxiliar as outras, seu propósito ficará mais claro adiante. 

In [11]:
def salvarArquivo(data, destino, log):
    #for registro in data:
    #print "escreveu no arquivo"
    data = json.loads(data)
    #print data
    order = 1
    for registro in data:
        try:            
            #print "vaiEscreverNoArquivo"
            #print registro
            destino.write("{\"order\":"+str(order)+",\"tweet\":"+json.dumps(registro)+"}\n")
            #print "Escreveu"
            #destino.flush()
            #print "flush"
            order += 1
        except:
            msg = "Ao Salvar o Arquivo"
            logError(log, msg)
            print ("Unexpected error:" + str(sys.exc_info()[0]))

Salva somente o texto e id de um tweet.

In [13]:
def customTweet(data, destino, ids):
    data = json.loads(data)
    cont = 0
    for row in data:
        destino.write(str(cont)+": "+json.dumps(row["text"])+'\n')
        ids.write(str(cont)+": "+json.dumps(str(row["id"]))+"\n")
        cont+=1
        #timeStamp.write(json.dumps(row["created_at"],'\n'))

Lembra da classe que criamos acima? Agora nós iremos usar uma função para instanciá-la e podermos preencher esse campo(identificação da coleta) no nosso banco ao chamar essa função.

In [14]:
def obterLastSnapshotId(collection):
    resultado = SnapshotIdSingleton.instance()
    try:
        resultado = resultado.getLastSnapShotIdFromMongo(collection)
    except:
        resultado = 0

    resultado += 1
    return resultado

# Salvando tweets

Aqui nós salvamos os tweets que coletarmos mais adiante no nosso banco de dados, com os seguintes campos:
* snapShotID: é a identificação da coleta realizada, então é comum ver mais de um tweet com esse registro
* snapshot_timestamp ou time_stamp: marca a hora em que a coleta foi realizada, útil para relacionar com a hora real de publicação dos tweets.
* bot_screen_name: obtém o @ do usuário logado/autenticado(um bot no caso).
* id_tweet: identificação do tweet.
* publisher_screen_name: pega o @ do perfil de quem publicou o tweet.
* impression_order: marca o número da coleta à qual o tweet pertence.
* retweet_count: quantidade de retweets numa publicação.
* favorite_count: quantidade de curtidas numa publicação.
* created_at_tweet: data de criação de um tweet, pode ser interessante cruzar esse dado com o timestamp ou impression order.
* text: texto de um tweet.

Essa função é uma adaptação de *salvarArquivo* para o projeto de modelagem da timeline, esses campos foram escolhidos assim porque foram pensados no ponto de vista de um *usuário* e o que a API do twitter mostra sobre a timeline dele.

In [16]:
#def salvarTweets(data, collection, destino, log, screen_name):
def salvarTweets(data, collection, collection2, log, screen_name):

    timestamp_snapshot = datetime.datetime.utcnow().strftime("%a %b %d %H:%M:%S +0000 %Y")
    #destino.write("\n"+timestamp_snapshot+"\t"+screen_name+"\n")
   
    data = json.loads(data)
    order = 1
    snapShotId = obterLastSnapshotId(collection)

    for registro in data:
        try:
            print ("111111111111111111111")
            #destino.write("{\"order\":"+str(order)+",\"tweet\":"+json.dumps(registro)+"}\n")
            completJSON = {'snapShotId':snapShotId, 'time_stamp':timestamp_snapshot, 'bot_screen_name':screen_name, 'impression_order':order,'tweet':json.dumps(registro)}

            print ("22222222222222222222")

            adjustedJSON = {'id_tweet':registro["id"],'bot_screen_name': screen_name,'publisher_screen_name':registro["user"]["screen_name"],'snapshot_id': snapShotId,'snapshot_timestamp':timestamp_snapshot,'impression_order':order,'retweet_count':registro["retweet_count"],'favorite_count':registro["favorite_count"],'created_at_tweet':registro["created_at"],'text':registro["text"]}

            #if (screen_name != ""):         
            #    adjustedJSON["bot_screen_name"] = screen_name

            collection.insert_one(adjustedJSON)
            collection2.insert_one(completJSON)
            print ("3333333333333333333333")

            order += 1

        except:
            msg = "Ao Salvar o Arquivo"
            logError(log, msg)
            print ("Unexpected error:" + str(sys.exc_info()[0]))


A função abaixo é similar à anterior, mas com um adendo, agora estamos interessados em salvar também os dados da timeline de um usuário especificado. 

In [17]:
#def salvarPublisherTweets(data, collection, destino, log):
def salvarPublisherTweets(data, collection, collection2, log):

    data = json.loads(data)

    for registro in data:
        try:

            #destino.write("{\"tweet\":"+json.dumps(registro)+"}\n")
            adjustedJSON = {'id_tweet':registro["id"],'publisher_screen_name':registro["user"]["screen_name"],'retweet_count':registro["retweet_count"],'favorite_count':registro["favorite_count"],'created_at_tweet':registro["created_at"]}

            collection.insert_one(adjustedJSON)
            collection2.insert_one(registro)

        except:
            msg = "Ao Salvar o Arquivo"
            logError(log, msg)
            print ("Unexpected error:" + str(sys.exc_info()[0]))

Adaptação do método customTweet para o projeto de modelagem da timeline.
A mudança é que a cada solicitação para salvar no arquivo eu incluo uma linha em branco, a hora UTC.

In [19]:
def customTweetAdapter(data, destino, ids,screen_name=""):

    destino.write("\n"+datetime.datetime.utcnow().strftime("%a %b %d %H:%M:%S +0000 %Y")+"\t"+screen_name+"\n")
    ids.write("\n"+datetime.datetime.utcnow().strftime("%a %b %d %H:%M:%S +0000 %Y")+"\t"+screen_name+"\n")
    customTweet(data, destino, ids)

## Pegando dados de um usuário especificado


Além de coletar a user_home(a do usuário autenticado)queremos os dados de uma timeline especificada pelo usuário, acima já fizemos uma função para salvar no banco, agora falta construir essa de coleta. 

In [20]:
def pegarTweetsDeUmUsuario(user, tweetTexto,destino, ids,maxTweets=1):

    if maxTweets > 200:
        maxTweets = 200
        print ("A quantidade maxima de tweets é 200")
    maxId = 0

    cont = 0
    while True:
        URL = "https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name="+user+"&count="+str(maxTweets)
        if (maxId > 0):
            URL +="&max_id=" + str(maxId)
        response,data = client.request(URL,"GET")

        print ("a",maxId)
        maxId = (json.loads(data)[-1]['id'] - 1)
        print ("b",maxId)

        if (cont >= 10):
            break;
        cont += 1

        customTweet(data, tweetTexto, ids)

        salvarArquivo(data, destino,log)

A função abaixo é uma adaptação da função pegarTweetsDeUmUsuario. A diferença é que se deseja pegar uma quantidade  (max de 200) de mensagens de uma timeline dado um intervalo de tempo. Esses dados são sempre os mais
recentes na timeline ainda que sejam repetidos a cada coleta. Por exemplo, se eu pego 5 tweets mais recentes e rodo novamente em 1 minuto e apenas uma nova mensagem chegou, pegarei os 5 mais recentes (o novo
e os outros 4 que já tinha coletado na rodada anterior).

Parâmetros:
* user: identifica perfil do twitter que está sendo 'monitorado'
* tweetTexto: é uma referência para um arquivo. Nele será apenas o texto do twitter.
* destino: arquivo para salvar os tweets coletados (todo o json).
* ids: é o arquivo para salvar os ids dos tweets.        
* collection: é a coleção para salvar os dados no mongo.
* intervalColeta: intervalo de coleta das mensagens em segundos.
* maxTweets: define a quantidade máxima de mensagens solicitadas a cada requisição. Pela API o máximo são de 200 mensagens.

In [23]:
#def pegarTweetsDeUmUsuarioQtdeIntervalo(user, tweetTexto,destino, ids, collection, intervalColeta=60, maxTweets=200):
def pegarTweetsDeUmUsuarioQtdeIntervalo(user, collection, collection2, intervalColeta=60, maxTweets=200):
    
    if maxTweets > 200:
        maxTweets = 200
        print ("A quantidade maxima de tweets é 200")
    sinceId = 0

    cont = 0
    while True:
        URL = "https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name="+user+"&count="+str(maxTweets)
        if (sinceId > 0):
            URL +="&since_id=" + str(sinceId)

        try:

            response,data = client.request(URL,"GET")

            sinceId = (json.loads(data)[0]['id']) #sempre coletar os posteriores a um certo id

            '''
            #apenas consulta 10x a API obtendo 10x a quantidade de tweets passada (se disponivel essa qtde).
            if (cont >= 10):
                break;
            cont += 1
            '''
            #customTweetAdapter(data, tweetTexto, ids)
            #salvarPublisherTweets(data, collection, destino,log)
            salvarPublisherTweets(data, collection, collection2, log)

        except:
            print ("quantidade maxima de tweets disponivel atingida. Ou seja, se vc pediu 10 e so tem 9, da erro.")
        finally:
            print ("finally")
            #time.sleep(intervalColeta) #dorme a cada iteracao
            time.sleep(10)

# Pegando a timeline de um usuário autenticado


A próxima função pega da home_timeline (linha do tempo do usuário com publicações dele e de quem ele segue).Pega a quantidade especificada em maxTweets (maximo de 200) dos tweets mais recentes na timeline.

No trabalho original nós queríamos pegar repetições (tweets que ele já viu, sem controlar se ele pega tweets que já foram recuperados antes - que seria feito com since_id, no caso). Na nossa pergunta seria sobre a timeline "como pegar apenas as mensagens mais recentes (20 últimas publicadas)?"... resolvido a explicação acima.


Parâmetros:
* tweetTexto - arquivo onde serão salvos os textos de cada tweet.
* destino - arquivo que salva tweets no formato JSON.
* ids - arquivo que salva o id dos tweets, esse arquivo é importante pois é possível obter o id de tweets de 30 dias atrás e é possível recuperar tweets antigos pelo id. 
* maxTweets - é usado para limitar o número de tweets a serem recuperados o valor padrão é 20 e o máximo é 200, mas atenção - max_id e maxTweets são parâmetros distintos.

É ainda possível usar outros parâmetros, confira a referência: https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-home_timeline
 


In [24]:
def pegarTimelineAuthUser(tweetTexto,destino,ids,maxTweets=100):

    '''
        Information:
            Response formats: JSON
            Require authentication
            Rate limits: 15 to each 15 minutes
    '''

    if maxTweets > 200:
        maxTweets = 200
        print ("A quantidade maxima de tweets é 200")
    maxId = 0

    cont = 0
    while True:
        URL = "https://api.twitter.com/1.1/statuses/home_timeline.json?count="+str(maxTweets)#+"&include_entities=false"#to exclude entites from json response
        if (maxId > 0):
            URL +="&max_id=" + str(maxId)
        response,data = client.request(URL,"GET")

        maxId = (json.loads(data)[-1]['id'] - 1)

        '''incluir testes do tamanho da janela'''

        if cont > 0:
            break       

        cont += 1

        customTweet(data, tweetTexto, ids)

        salvarArquivo(data, destino,log)


Uma adaptação da função anterior, nós iremos chamá-la na main para começar a coleta dos tweets e inserção no banco de dados.

In [ ]:
#def pegarTimelineAuthUserProjetoTimeline(tweetTexto,destino,ids,collection, screen_name,intervalColeta=60,maxTweets=200):
def pegarTimelineAuthUserProjetoTimeline(collection, collection2, screen_name,intervalColeta=60,maxTweets=200):

    if maxTweets > 200:
        maxTweets = 200
        print "A quantidade maxima de tweets é 200"
    #maxId = 0

    #cont = 0
    while True:
        URL = "https://api.twitter.com/1.1/statuses/home_timeline.json?count="+str(maxTweets)#+"&include_entities=false"#to exclude entites from json response
        #if (maxId > 0):
        #    URL +="&max_id=" + str(sinceId)

        try: #nao precisa, mas deixei porque estava dando erro quando estava usando maxId
            response,data = client.request(URL,"GET")
            #maxId = (json.loads(data)[-1]['id']-1)#uma lista de tweets (padrao e 200 no param maxTweets) sao obtidos. Aqui pego o id do ultimo tweet da lista. Entao coleto os mais antigos, antes dele na proxima iteracao.

            '''incluir testes do tamanho da janela

            if cont > 0:
                break       
            
            cont += 1
            '''
            #customTweetAdapter(data, tweetTexto, ids, screen_name)
            #salvarTweets(data, collection, destino, log, screen_name)
            salvarTweets(data, collection, collection2, log, screen_name)
            #contExc = 0
        except:
           print "exception"
           #contExc += 1
            
        finally:
            print "finally"           
            time.sleep(intervalColeta)

A que vamos ver agora é complementar à próxima, basicamente ela vai fazer uma requisição para pegar tweets antigos baseados no id.

In [ ]:
def pegarTweets(ids, destino, log):

    listaIds = ""
    URL = "https://api.twitter.com/1.1/statuses/lookup.json?id="
    for j in ids:
        listaIds += str(j)+","
    
    listaIds = listaIds[:-1]#remover ultima virgula
    URL += listaIds
    #print "vaiEnviarReqisição"
    response,data = client.request(URL,"GET")

    #print "enviouRequisiçãoVaiSalvar"
    salvarArquivo(data, destino,log)
    #print "Salvou"

A função abaixo está comentada na main, mas se você quiser usá-la, o que ela faz é coletar os tweets antigos salvos por ID naquele arquivo anterior da função *pegarTimelineAuthUser*.

In [ ]:
def coletarTweetsAntigos(arqIDs, destino, log):
    ids = []

    #arqIDs = ["577982721596723200"]

    for i in arqIDs:
        #i = i.replace("\n","")
        #i = i.replace("\r","")
        i = re.sub("[^0-9]","",i)


        ids.append(i)
        
            #URL = "https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=tcruzfranca&count=1"
        if len(ids) == 100:
            try:
                print "pegaTweets"
                pegarTweets(ids,destino,log)
                print "depoisPegarTweets"
                time.sleep(5)                        
            except:
                msg = "Erro ao coletar."                
                #logErro(log,msg)
                print "Unexpected error:"+ str(sys.exc_info()[0])
                print "dormiu"
                time.sleep(60*15)
                pegarTweets(ids,destino,log)
            finally:
                ids = []

    else:
        pegarTweets(ids,destino, log)

### As duas próximas funções serão usadas apenas para auxiliar a main().


É necessário obter o screen_name(@ do seu usuário) para salvar os dados no banco mongo.

In [ ]:
def getScreenNameAuthUser():

    URL = "https://api.twitter.com/1.1/account/settings.json"
    response, data = client.request(URL,"GET")

    screen_name = json.loads(data)
    screen_name = screen_name["screen_name"]
    return screen_name

Agora nós vamos validar as suas credenciais, aquelas que foram fornecidas anteriormente ao arquivo "credentials.txt".

In [2]:
def getCredentials():
    
    arq = open("credentials.txt")
    
    credentials = {}

    for i in arq:
        i = i.replace("\n","")
        i = i.replace(" ","")
        aux = i.split("=")
        key = aux[0]
        value = aux[1]
        
        credentials.update({key:value})

    return credentials

*Dependências da função main: oauth2 e codecs

# Função main

###### pegarTimelineAuthUserProjetoTimeline: pega conteúdo da timeline do usuário proprietário das credenciais.
###### pegarTweetsDeUmUsuarioQtdeIntervalo: pega conteúdo da timeline de um usuário específico(o que ele publicou).
  
  
  Será perguntado a você(no terminal da sua máquina) qual o intervalo entre as coletas(em segundos!!!) e a quantidade de tweets por requisição independentemente de qual função for escolhida. O padrão é de 30 tweets a cada 1500 segundos.
  
  E pronto! Agora basta deixar o programa rodando, cuidado apenas com quedas de energia ou falhas em conexão de rede. É de extrema importância monitorar as coletas para não perder dados importantes.

In [1]:
if __name__ == "__main__":
  
    credenciais = getCredentials()

    Consumer_key = credenciais['Consumer_key']
    Consumer_secret = credenciais['Consumer_secret']
    Access_token = credenciais['Access_token']
    Access_token_secret = credenciais['Access_token_secret']

    consumer = oauth.Consumer(key=Consumer_key, secret=Consumer_secret)
    access_token = oauth.Token(key=Access_token, secret=Access_token_secret)
    client = oauth.Client(consumer, access_token)


    arquivo = raw_input("Forneca o nome do banco onde serao salvos os dados da coleta:")
    #print ("Esse mesmo nome tambem sera usado para salvar todos os dados em um arquivo.")

    #substitua os nomes dos arquivos
    #salvarTweet = arquivo+".txt"
    #salvarApenasIdsTweets = arquivo+"_ids.txt"
    #salvarApenasTextoTweets = arquivo+"_texto.txt"
    salvarLogErros = arquivo+"_log.txt"

    #destino = codecs.open(salvarTweet, "a", "utf-8")
    #tweets_ids = codecs.open(salvarApenasIdsTweets, "a", "utf-8")
    #tweets_texto= codecs.open(salvarApenasTextoTweets, "a", "utf-8")
    log = open(salvarLogErros,"a")

    mongo = pymongo.MongoClient()
    db = mongo[arquivo]

    #print ("Os arquivos:"+salvarTweet+','+salvarApenasIdsTweets+','+salvarApenasTextoTweets+','+salvarLogErros+" foram criados")

    try:
        intervaloColeta = input("Qual o intervalo de coleta (em segundos)? (default 1500s)") #3600
        qtdeTweetsPorRequisicao = input("Qual a quantidade de tweets deseja por requisicao (min = 1 e max = 200)?") #30
    except:
        intervaloColeta = 3600
        qtdeTweetsPorRequisicao = 30

    resp = raw_input("Deseja coletar da timeline do usuário dono das credenciais fornecidas? (S-sim, n-nao)")

    if (resp != 'n'):
        screen_name = getScreenNameAuthUser()
        collection = db[screen_name]
        collection2 = db[screen_name+"_completeJSON"]
        #collection.find_one()
        #screen_name is the name of the logged user
        #pegarTimelineAuthUserProjetoTimeline(tweets_texto, destino, tweets_ids, collection, screen_name, intervaloColeta, qtdeTweetsPorRequisicao)
        pegarTimelineAuthUserProjetoTimeline(collection, collection2, screen_name, intervaloColeta, qtdeTweetsPorRequisicao)
    else: 
        userName = raw_input("Qual nome do usuario que voce deseja coletar dados da timeline (sem '@') ?")
        userName = userName.replace("@","")
        collection = db[userName]
        collection2 = db[userName+"_completeJSON"]
        #pegarTweetsDeUmUsuarioQtdeIntervalo(userName, tweets_texto,destino,tweets_ids, collection, intervaloColeta,qtdeTweetsPorRequisicao)
        pegarTweetsDeUmUsuarioQtdeIntervalo(userName, collection, collection2, intervaloColeta,qtdeTweetsPorRequisicao)

    print("A coleta esta sendo realizada. Sente, tome um cafe e trabalhe em outra coisa. Ou coloque outra coleta pra rodar, caso deseje. =D")
    #coletarTweetsAntigos(arq, destino,log)#coleta por ids que estao no arquivo idsTweets


    destino.close()
    tweets_texto.close()
    log.close()


NameError: name 'getCredentials' is not defined

# Executando localmente

Para executar o programa digite no seu terminal: python pegarTweetsProjTimeline.py. 

# Carregando o dataset

A partir deste ponto você pode seguir como quiser, fica aqui apenas uma sugestão de como carregar o dataset.

Se você é minimamente familiar com a linguagem R, abra o Rstudio e digite no terminal dele os comandos abaixo.

> install.packages("jsonlite")

> library(jsonlite)

> nome_do_arquivo <- '/SeuLocalAqui/seuArquivo.json'

> dataset <-jsonlite::stream_in(textConnection(readLines(nome_do_arquivo, n=1000)),verbose=F)

Onde na última linha(dataset) o argumento "n=1000" do método readLines() se refere ao número máximo de linhas que você deseja visualizar, o que pode ser útil caso o dataset seja muito pesado e você não pretenda carregar tudo de uma vez.